# Анализ эксперимента по подстройке коэффициентов Kp и Ki (с включенной защитой от падения напряжения) от 06 ноября 2025


In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

EXPERIMENT_DIR = Path("../experiments/td3_train_async/2025-11-06_14-26-53")
ENV_LOG_PATH = EXPERIMENT_DIR / "env_logs" / "env.log"
TRAIN_LOG_PATH = EXPERIMENT_DIR / "train_async.log"
TRAIN_LOGS_PATH = EXPERIMENT_DIR / "train_logs" / "train.log"
CONNECTION_LOG_PATH = EXPERIMENT_DIR / "connection_logs" / "connection.log"

## Анализ логов окружения


In [ ]:
def parse_env_logs(file_path):
    pattern = re.compile(
        r"step=(?P<step>\d+)\s+"
        r"phase=(?P<phase>\w+)\s+"
        r"block_step=(?P<block_step>\w+)\s+"
        r"kp=(?P<kp>-?\d+\.\d+)\s+"
        r"ki=(?P<ki>-?\d+\.\d+)\s+"
        r"kd=(?P<kd>-?\d+\.\d+)\s+"
        r"delta_kp_norm=(?P<delta_kp_norm>-?\d+\.\d+)\s+"
        r"delta_ki_norm=(?P<delta_ki_norm>-?\d+\.\d+)\s+"
        r"error_mean=(?P<error_mean>-?\d+\.\d+)\s+"
        r"error_std=(?P<error_std>-?\d+\.\d+)\s+"
        r"error_mean_norm=(?P<error_mean_norm>-?\d+\.\d+)\s+"
        r"error_std_norm=(?P<error_std_norm>-?\d+\.\d+)\s+"
        r"reward=(?P<reward>-?\d+\.\d+)"
    )
    
    rows = []
    with open(file_path, 'r') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            match = pattern.match(line)
            if match:
                rows.append({
                    'Step': int(match.group('step')),
                    'Kp': float(match.group('kp')),
                    'Ki': float(match.group('ki')),
                    'Kd': float(match.group('kd')),
                    'Delta Kp': float(match.group('delta_kp_norm')),
                    'Delta Ki': float(match.group('delta_ki_norm')),
                    'Error mean': float(match.group('error_mean')),
                    'Error std': float(match.group('error_std')),
                    'Error mean norm': float(match.group('error_mean_norm')),
                    'Error std norm': float(match.group('error_std_norm')),
                    'Reward': float(match.group('reward'))
                })
    
    return pd.DataFrame(rows)

env_df = parse_env_logs(ENV_LOG_PATH)
print(f"Загружено {len(env_df)} записей из логов окружения")

In [ ]:
print("=== Статистика по данным окружения ===")
print(env_df.describe())

In [ ]:
neural_network_step = (2500 + 150) * 200 + 1000

columns_to_plot = [col for col in env_df.columns if col != 'Step']

for col in columns_to_plot:
    plt.figure(figsize=(10, 4))
    plt.plot(env_df['Step'], env_df[col], alpha=0.8, linewidth=0.8, label=col)
    
    plt.axvline(x=neural_network_step, color='red', linestyle='--', linewidth=2, 
                label=f'Switch to NN')
    
    plt.title(f'{col} over Steps')
    plt.xlabel('Step')
    plt.ylabel(col)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(env_df['Step'], env_df["Error std"], alpha=0.8, linewidth=0.8, label=col)
plt.xlim(left=neural_network_step)
plt.ylim(top=100)

plt.title('Error std over Steps (NN)')
plt.xlabel('Step')
plt.ylabel(col)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
cur_ax = axes[0]
sns.scatterplot(x=env_df['Kp'], y=env_df['Error std'], alpha=0.6, ax=cur_ax)
cur_ax.set_xlabel('Kp')
cur_ax.set_ylabel('Error Std')
cur_ax.grid(True, alpha=0.3)

cur_ax = axes[1]
sns.scatterplot(x=env_df['Kp'], y=env_df['Error mean'], alpha=0.6, ax=cur_ax, color='orange')
cur_ax.set_xlabel('Kp')
cur_ax.set_ylabel('Error mean')
cur_ax.grid(True, alpha=0.3)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))
cur_ax = axes[0]
sns.scatterplot(x=env_df['Ki'], y=env_df['Error std'], alpha=0.6, ax=cur_ax)
cur_ax.set_xlabel('Ki')
cur_ax.set_ylabel('Error Std')
cur_ax.grid(True, alpha=0.3)

cur_ax = axes[1]
sns.scatterplot(x=env_df['Ki'], y=env_df['Error mean'], alpha=0.6, ax=cur_ax, color='orange')
cur_ax.set_xlabel('Ki')
cur_ax.set_ylabel('Error mean')
cur_ax.grid(True, alpha=0.3)

In [ ]:
corr_matrix = env_df[['Kp', 'Error mean', 'Error std', 'Reward']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f");

In [ ]:
corr_matrix = env_df[['Ki', 'Error mean', 'Error std', 'Reward']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f");

## Анализ процесса обучения


In [ ]:
def parse_loss_logs(file_path):
    pattern = re.compile(
        r"step=(?P<step>\d+)\s+Loss/Critic=(?P<critic_loss>-?\d+\.\d+)\s+Loss/Actor=(?P<actor_loss>-?\d+\.\d+)"
    )
    
    rows = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            match = pattern.match(line)
            if match:
                rows.append({
                    'step': int(match.group('step')),
                    'critic_loss': float(match.group('critic_loss')),
                    'actor_loss': float(match.group('actor_loss'))
                })
    
    return pd.DataFrame(rows)

loss_df = parse_loss_logs(TRAIN_LOGS_PATH)
print(f"Загружено {len(loss_df)} записей из логов потерь")
print(f"Диапазон шагов обучения: {loss_df['step'].min()} - {loss_df['step'].max()}")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

axes[0].plot(loss_df['step'], loss_df['critic_loss'], 'b-')
axes[0].set_title('Critic Loss')
axes[0].set_xlabel('Step')
axes[0].grid(True)

axes[1].plot(loss_df['step'], loss_df['actor_loss'], 'r-')
axes[1].set_title('Actor Loss')
axes[1].set_xlabel('Step')
axes[1].grid(True)

plt.tight_layout()

## Анализ состояния установки


In [ ]:
def parse_connection_logs(file_path):
    send_pattern = re.compile(r"SEND: kp=(?P<kp>-?\d+\.\d+) ki=(?P<ki>-?\d+\.\d+) kd=(?P<kd>-?\d+\.\d+) control_min=(?P<control_min>\d+) control_max=(?P<control_max>\d+)")
    read_pattern = re.compile(r"READ: process_variable=(?P<process_variable>-?\d+\.\d+) control_output=(?P<control_output>-?\d+\.\d+)")
    
    send_rows = []
    read_rows = []
    step = 0
    
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            
            send_match = send_pattern.match(line)
            if send_match:
                send_rows.append({
                    'step': step,
                    'type': 'SEND',
                    'kp': float(send_match.group('kp')),
                    'ki': float(send_match.group('ki')),
                    'kd': float(send_match.group('kd')),
                    'control_min': int(send_match.group('control_min')),
                    'control_max': int(send_match.group('control_max'))
                })
                step += 1
            
            read_match = read_pattern.match(line)
            if read_match:
                read_rows.append({
                    'step': step - 1,
                    'type': 'READ',
                    'process_variable': float(read_match.group('process_variable')),
                    'control_output': float(read_match.group('control_output'))
                })
    
    connection_df = pd.DataFrame(send_rows)
    read_df = pd.DataFrame(read_rows)
    
    if not connection_df.empty and not read_df.empty:
        connection_df = connection_df.merge(read_df[['step', 'process_variable', 'control_output']], 
                                          on='step', how='left')
    
    return connection_df

connection_df = parse_connection_logs(CONNECTION_LOG_PATH)
print(f"Загружено {len(connection_df)} записей из логов соединения")
print(f"Диапазон шагов: {connection_df['step'].min()} - {connection_df['step'].max()}")


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(connection_df['step'], connection_df['process_variable'], 'b-', alpha=0.7, linewidth=0.8)
plt.axhline(y=1200, color='r', linestyle='--', label='Setpoint')
if neural_network_step is not None:
    plt.axvline(x=neural_network_step, color='red', linestyle='--', linewidth=2, 
                label=f'Switch to NN')
plt.title('Process Variable')
plt.xlabel('Step')
plt.ylabel('Process Variable')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(connection_df['step'], connection_df['control_output'], 'g-', alpha=0.7, linewidth=0.8)
if neural_network_step is not None:
    plt.axvline(x=neural_network_step, color='red', linestyle='--', linewidth=2, 
                label=f'Switch to NN')
plt.title('Control Output')
plt.xlabel('Step')
plt.ylabel('Control Output')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(connection_df['step'], connection_df['kp'], 'r-', alpha=0.7, linewidth=0.8, label='Kp')
plt.plot(connection_df['step'], connection_df['ki'], 'g-', alpha=0.7, linewidth=0.8, label='Ki')
plt.plot(connection_df['step'], connection_df['kd'], 'b-', alpha=0.7, linewidth=0.8, label='Kd')
if neural_network_step is not None:
    plt.axvline(x=neural_network_step, color='red', linestyle='--', linewidth=2, 
                label=f'Switch to NN')
plt.title('PID Coefficients')
plt.xlabel('Step')
plt.ylabel('Coefficient Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
